#### **PROJECT TITLE :** `AI-Powered Medical Image Analysis Tool for Chest (X-Ray and CT-Scan) Diseases Detection`

<br>
<br>

### Component : `Model Deployment + Testing on Random Images`

In [1]:
# Let us first import all the necessary libraries required for this project

import tensorflow as tf
import torch
import cv2
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from PIL import Image

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc


# Later on, as per requirement, more libraries wil be imported

<br>
<br>

### <b>STEP - 1 : Pre-Processing Functions</b>

In [3]:
def adjust_brightness_contrast(image, alpha=1.2, beta=50):
    """
    Adjusting brightness and contrast of the image.
    Parameters:
        - image: Input image (numpy array).
        - alpha: Contrast control [1.0-3.0].
        - beta: Brightness control [0-100].
    Returns:
        - Adjusted image.
    """
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def apply_histogram_equalization(image):
    """Applying histogram equalization to enhance contrast."""
    channels = cv2.split(image)
    eq_channels = [cv2.equalizeHist(ch) for ch in channels]
    return cv2.merge(eq_channels)

def apply_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    """Applying CLAHE for local contrast enhancement."""
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    channels = cv2.split(image)
    clahe_channels = [clahe.apply(ch) for ch in channels]
    return cv2.merge(clahe_channels)

def apply_gaussian_blur(image, kernel_size=(3, 3)):
    """Applying Gaussian blur for denoising."""
    return cv2.GaussianBlur(image, kernel_size, 0)

def apply_sharpening(image):
    """Applying edge enhancement using a sharpening filter."""
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def normalize_image(image):
    """Normalizing the image to zero mean and unit variance."""
    image = (image - np.mean(image)) / np.std(image)
    return image

def resize_image(image, width, height):
    """Resizing the image to the desired dimensions with anti-aliasing."""
    return cv2.resize(image, (width, height), interpolation=cv2.INTER_CUBIC)

<br>
<br>

### <b>STEP - 2 : Pre-processing Pipeline</b>

In [4]:
def preprocess_single_image(img_path, img_height=224, img_width=224):
    """
    Preprocessing a single image as per the training pipeline.
    """
    # Loading image in BGR format
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError(f"Image not found: {img_path}")

    # Converting BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Applying preprocessing steps
    image = apply_histogram_equalization(image)
    image = apply_clahe(image)
    image = apply_gaussian_blur(image)
    image = apply_sharpening(image)
    image = adjust_brightness_contrast(image, alpha=1.2, beta=50)

    # Resizing and normalize
    image = resize_image(image, img_width, img_height)
    image = normalize_image(image)

    # Converting to PIL image and applyinh transformations
    image = Image.fromarray(image.astype(np.uint8))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image_tensor = transform(image).unsqueeze(0)  # Adding batch dimension
    return image_tensor

<br>
<br>

### <b>STEP - 3 : Loading the Model</b>

In [2]:
# Detecting GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
from torchvision.models import densenet121
from torchvision.transforms import Resize

# Simplified ViT-like transformer module
class SimpleViT(nn.Module):
    def __init__(self, input_dim, num_heads, mlp_dim, num_layers):
        super(SimpleViT, self).__init__()
        # Reduced TransformerEncoder layer complexity
        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=input_dim,
                nhead=num_heads,
                dim_feedforward=mlp_dim,
                dropout=0.1
            ) for _ in range(num_layers)
        ])

    def forward(self, x):
        # Flattening the spatial dimensions
        B, C, H, W = x.shape
        x = x.flatten(2).permute(2, 0, 1)  # Reshaping for transformer
        for block in self.transformer_blocks:
            x = block(x)
        x = x.permute(1, 2, 0).reshape(B, C, H, W)  # Restoring the original shape
        return x


# Adjusted Hybrid DenseNet + Simplified ViT Architecture
class LightweightHybridDenseNetViT(nn.Module):
    def __init__(self):
        super(LightweightHybridDenseNetViT, self).__init__()

        # Loading a lighter DenseNet backbone
        self.densenet = densenet121(pretrained=False)  # Base DenseNet backbone

        # Reducing the output channels from DenseNet to smaller dimensions
        self.conv_reduce = nn.Conv2d(1024, 64, kernel_size=1)  # Fewer channels

        # ViT processing module with reduced complexity
        self.vit = SimpleViT(input_dim=64, num_heads=2, mlp_dim=128, num_layers=1)

        # Task-specific classification heads
        self.fc_pneumonia = nn.Linear(64, 1)  # Binary classification (Pneumonia)
        self.fc_tuberculosis = nn.Linear(64, 1)  # Binary classification (Tuberculosis)
        self.fc_lung_cancer = nn.Linear(64, 4)  # Multi-class output (Lung Cancer)

    def forward(self, x):
        # Extracting DenseNet features
        x = self.densenet.features(x)  # Extracting DenseNet feature maps
        x = self.conv_reduce(x)  # Reducing the number of feature channels

        # Passing through simplified ViT module
        x = self.vit(x)

        # Applying Global Average Pooling (GAP)
        x = x.mean(dim=[2, 3])  # Pooling across spatial dimensions

        # Task-specific classification
        pneumonia_output = torch.sigmoid(self.fc_pneumonia(x))  # Binary sigmoid output
        tuberculosis_output = torch.sigmoid(self.fc_tuberculosis(x))  # Binary sigmoid output
        lung_cancer_output = self.fc_lung_cancer(x)  # Multi-class logits

        return pneumonia_output, tuberculosis_output, lung_cancer_output

In [7]:
# Loading the saved model
img_size = 224  # Matching the dimensions used during training
patch_size = 8


model = LightweightHybridDenseNetViT().to(device)

model.load_state_dict(torch.load("model_FINAL.pth", map_location=device))  # Mapping to the correct device
model.to(device)  # Moving the model to GPU/CPU
model.eval()  # Setting to evaluation mode

e:\anaconda3\envs\CVIP_ENV\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\anaconda3\envs\CVIP_ENV\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\dpsvn\AppData\Local\Temp\ipykernel_31420\2949034845.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be fl

LightweightHybridDenseNetViT(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, e

<br>
<br>

### <b>STEP - 4 : Preprocessing the Image and Moving to GPU</b>

In [8]:
image_tensor = preprocess_single_image("Deploy-Test Images/642x361_SLIDE_4_What_Does_Lung_Cancer_Look_Like.jpg")  # Path to the random image
image_tensor = image_tensor.to(device)  # Moving the input tensor to the same device

<br>
<br>

### <b>STEP - 5 : Performing Inference on GPU</b>

In [9]:
# Performing inference
with torch.no_grad():
    pneumonia_output, tb_output, lung_cancer_output = model(image_tensor)

# Converting outputs to probabilities
pneumonia_prob = pneumonia_output.item()  # Sigmoid ensures output in [0, 1]
tb_prob = tb_output.item()
lung_cancer_probs = F.softmax(lung_cancer_output, dim=1).squeeze().tolist()

# Class names for lung cancer
lung_cancer_classes = [
    "adenocarcinoma_left.lower.lobe",
    "large.cell.carcinoma_left.hilum",
    "NORMAL",
    "squamous.cell.carcinoma_left.hilum"
]

# Printing results
print(f"Pneumonia Probability: {pneumonia_prob:.4f}")
print(f"TB Probability: {tb_prob:.4f}")

# Printing lung cancer probabilities with class names
print("Lung Cancer Probabilities:")
for class_name, prob in zip(lung_cancer_classes, lung_cancer_probs):
    print(f"    {class_name}: {prob:.4f}")

Pneumonia Probability: 0.9692
TB Probability: 0.8210
Lung Cancer Probabilities:
    adenocarcinoma_left.lower.lobe: 0.0384
    large.cell.carcinoma_left.hilum: 0.0717
    NORMAL: 0.7883
    squamous.cell.carcinoma_left.hilum: 0.1015


<br>
<br>

### <b>STEP - 6 : Taking a directory of images into consideration</b>

In [10]:
# Directory containing the test images

image_folder = "Deploy-Test Images"

In [11]:
# Looping through the folder, preprocessing each image, and running inference
results = []
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    if not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Ensuring it's an image file
        continue

    # Preprocessing the image
    image_tensor = preprocess_single_image(image_path)
    image_tensor = image_tensor.to(device)

    # Performing inference
    with torch.no_grad():
        pneumonia_output, tb_output, lung_cancer_output = model(image_tensor)

    # Converting outputs to probabilities
    pneumonia_prob = pneumonia_output.item()
    tb_prob = tb_output.item()
    lung_cancer_probs = F.softmax(lung_cancer_output, dim=1).squeeze().tolist()

    # Appending the results for the current image
    results.append({
        "image_name": image_name,
        "pneumonia_prob": pneumonia_prob,
        "tb_prob": tb_prob,
        "lung_cancer_probs": lung_cancer_probs
    })

    lung_cancer_classes = [
    "adenocarcinoma_left.lower.lobe",
    "large.cell.carcinoma_left.hilum",
    "NORMAL",
    "squamous.cell.carcinoma_left.hilum"
    ]

    # Printing the results
    print(f"\nImage: {image_name}")
    print(f"  Pneumonia Probability: {pneumonia_prob:.4f}")
    print(f"  TB Probability: {tb_prob:.4f}")

    # Printing lung cancer probabilities with class names
    print("  Lung Cancer Probabilities:")
    for class_name, prob in zip(lung_cancer_classes, lung_cancer_probs):
        print(f"    {class_name}: {prob:.4f}")


Image: 4994014ef5c834e4803541aa1dc874_big_gallery.jpg
  Pneumonia Probability: 0.9534
  TB Probability: 0.9138
  Lung Cancer Probabilities:
    adenocarcinoma_left.lower.lobe: 0.0689
    large.cell.carcinoma_left.hilum: 0.1746
    NORMAL: 0.4685
    squamous.cell.carcinoma_left.hilum: 0.2880

Image: 4ceeb362df213ccf2c0b4d6388bba1_gallery.jpg
  Pneumonia Probability: 0.9514
  TB Probability: 0.7690
  Lung Cancer Probabilities:
    adenocarcinoma_left.lower.lobe: 0.0709
    large.cell.carcinoma_left.hilum: 0.1399
    NORMAL: 0.5900
    squamous.cell.carcinoma_left.hilum: 0.1993

Image: 642x361_SLIDE_4_What_Does_Lung_Cancer_Look_Like.jpg
  Pneumonia Probability: 0.9692
  TB Probability: 0.8210
  Lung Cancer Probabilities:
    adenocarcinoma_left.lower.lobe: 0.0384
    large.cell.carcinoma_left.hilum: 0.0717
    NORMAL: 0.7883
    squamous.cell.carcinoma_left.hilum: 0.1015

Image: Chest_radiograph_of_miliary_tuberculosis_2.jpg
  Pneumonia Probability: 0.9638
  TB Probability: 0.0253
  Lun